In [1]:
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib as plt
import pandas as pd
import torch.utils.data as data_utils
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import pytorch_lightning as pl

In [2]:
data = pd.read_csv(r"/content/drive/MyDrive/DistanceClassifier/data2/glass.csv")  # 1
df = data


In [3]:
dum_df = pd.get_dummies(df, columns=["Type"])
new_df = dum_df#.sample(frac = 1).reset_index(drop = True)
new_df.columns

Index(['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type_1', 'Type_2',
       'Type_3', 'Type_5', 'Type_6', 'Type_7'],
      dtype='object')

In [4]:
Y_cols = ['Type_1', 'Type_2','Type_3', 'Type_5', 'Type_6', 'Type_7']

In [5]:
X = new_df.drop(Y_cols, axis=1).values  # 2

y = new_df[Y_cols].copy()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
display(X_train)
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)

array([[ 1.51652, 13.56   ,  3.57   , ...,  7.96   ,  0.     ,  0.     ],
       [ 1.51748, 12.86   ,  3.56   , ...,  8.38   ,  0.     ,  0.17   ],
       [ 1.52058, 12.85   ,  1.61   , ...,  9.7    ,  0.24   ,  0.51   ],
       ...,
       [ 1.51763, 12.61   ,  3.59   , ...,  8.5    ,  0.     ,  0.     ],
       [ 1.52177, 13.75   ,  1.01   , ..., 11.14   ,  0.     ,  0.     ],
       [ 1.51508, 15.15   ,  0.     , ...,  8.34   ,  0.63   ,  0.     ]])

In [6]:
y.values.dtype

dtype('uint8')

In [7]:
y_train

,Type_1,Type_2,Type_3,Type_5,Type_6,Type_7
119,0,1,0,0,0,0
13,1,0,0,0,0,0
174,0,0,0,1,0,0
195,0,0,0,0,0,1
22,1,0,0,0,0,0
...,...,...,...,...,...,...
166,0,0,0,1,0,0
146,0,0,1,0,0,0
14,1,0,0,0,0,0
130,0,1,0,0,0,0


In [8]:
X_train

tensor([[ 1.5165, 13.5600,  3.5700,  ...,  7.9600,  0.0000,  0.0000],
        [ 1.5175, 12.8600,  3.5600,  ...,  8.3800,  0.0000,  0.1700],
        [ 1.5206, 12.8500,  1.6100,  ...,  9.7000,  0.2400,  0.5100],
        ...,
        [ 1.5176, 12.6100,  3.5900,  ...,  8.5000,  0.0000,  0.0000],
        [ 1.5218, 13.7500,  1.0100,  ..., 11.1400,  0.0000,  0.0000],
        [ 1.5151, 15.1500,  0.0000,  ...,  8.3400,  0.6300,  0.0000]],
       dtype=torch.float64)

In [9]:
def df_to_dict(y_numpy):
  y_numpy = y_numpy.astype(float)
  one_hot_dict = {}
  cols = []
  for col in Y_cols:
    cols.append(y_numpy[col].values)
  for i in range(len(cols)):
    one_hot_dict[i] = torch.tensor(cols[i])
  return one_hot_dict

In [10]:
y_train = df_to_dict(y_train)
y_test = df_to_dict(y_test)

In [11]:
y_train[1]

tensor([1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
        1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 0.], dtype=torch.float64)

In [12]:
def sigmoid(x, c1, c2):
  e = 2.71828
  return 1 / (1 + torch.exp(e, (-1 * c1) * (x - c2)))

class SigmoidRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SigmoidRegression, self).__init__()
        c1 = torch.randn(1, requires_grad = True)
        c2 = torch.randn(1, requires_grad = True)
        self.sequential = torch.nn.Sequential(torch.nn.Linear(input_dim, output_dim), 
                                              torch.nn.Sigmoid())
        self.linear = torch.nn.Linear(input_dim, output_dim)
    def forward(self, x):
        return self.sequential(x)

In [13]:
batch_size = 20
n_iters = 5000
input_dim = 9
output_dim = 1

#model = SigmoidRegression(input_dim, output_dim)
#optimizer = torch.optim.SGD(model.parameters(), lr= 0.001, weight_decay = 0.01)

In [14]:
sigmoids = []
for i in range(len(Y_cols)):
    sigmoids.append((SigmoidRegression))

In [18]:
def train(iter, model, X_train, y_train, loss_func):
  n_iterations = iter
  optimizer = torch.optim.SGD(model.parameters(), lr= 0.001, weight_decay = 0.01)
  loss_f = loss_func
  for i in range(1, n_iterations + 1):
      Z = model(X_train.float())  # 1
      print(Z)
      loss = loss_func(Z.float(), y_train.unsqueeze(1).float())  # 2
      optimizer.zero_grad()  # 3
      loss.backward()  # 4
      optimizer.step()  # 5
      if i == 1 or i % 10000 == 0:
        print("Loss at iteration {}: {}".format(i, loss))
  print("")
      
  # test_predictions = torch.argmax(
  #     torch.softmax(model(X_test), 1), axis=1  # 6
  # )
  # test_accuracy = float(sum(test_predictions == y_test)) / y_test.shape[0]
  # print("\nFinal Test Accuracy: {}".format(test_accuracy))

In [16]:
for i in range(len(sigmoids)):
  train(100000, 
        sigmoids[i](input_dim, output_dim),
        X_train,
        y_train[i],
        torch.nn.BCELoss())

Loss at iteration 1: 6.3924479484558105
Loss at iteration 10000: 0.4813907742500305

Loss at iteration 1: 9.811943054199219
Loss at iteration 10000: 0.6099000573158264

Loss at iteration 1: 7.408224582672119
Loss at iteration 10000: 0.20133303105831146

Loss at iteration 1: 0.5174504518508911
Loss at iteration 10000: 0.1606021672487259

Loss at iteration 1: 2.4098896980285645
Loss at iteration 10000: 0.13114841282367706

Loss at iteration 1: 85.90604400634766
Loss at iteration 10000: 85.90604400634766



In [19]:
  train(10000, 
        sigmoids[1](input_dim, output_dim),
        X_train,
        y_train[1],
        torch.nn.BCELoss())

Streaming output truncated to the last 5000 lines.
        [0.3256],
        [0.3592],
        [0.5372],
        [0.3153],
        [0.3150],
        [0.3615],
        [0.3318],
        [0.3227],
        [0.4216],
        [0.3453],
        [0.4346],
        [0.2870],
        [0.1962],
        [0.3561],
        [0.3769],
        [0.3403],
        [0.2925],
        [0.3356],
        [0.3218],
        [0.3702],
        [0.3311],
        [0.3021],
        [0.3137],
        [0.3279],
        [0.3443],
        [0.4717],
        [0.3489],
        [0.2115],
        [0.4395],
        [0.3579],
        [0.3260],
        [0.3490],
        [0.3141],
        [0.3189],
        [0.3366],
        [0.3054],
        [0.3168],
        [0.6297],
        [0.5389],
        [0.3203],
        [0.5256],
        [0.3032],
        [0.2994],
        [0.3510],
        [0.3024],
        [0.3287],
        [0.3582],
        [0.3668],
        [0.3381],
        [0.3551],
        [0.3006],
        [0.3930],
        [0.29

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
        [0.3756],
        [0.3999],
        [0.4089],
        [0.3690],
        [0.3708],
        [0.2277],
        [0.3699],
        [0.3604],
        [0.2849],
        [0.3870],
        [0.4173],
        [0.2630],
        [0.2181],
        [0.3832],
        [0.2952],
        [0.3900],
        [0.3404],
        [0.3602],
        [0.2830],
        [0.4016],
        [0.3845],
        [0.3563],
        [0.3655],
        [0.3645],
        [0.3994],
        [0.3366],
        [0.3733],
        [0.1297],
        [0.3001],
        [0.3888],
        [0.3814],
        [0.4022],
        [0.3664],
        [0.3678],
        [0.3940],
        [0.3651],
        [0.3833],
        [0.5286],
        [0.4258],
        [0.3650],
        [0.4205],
        [0.3447],
        [0.3334],
        [0.3923],
        [0.3332],
        [0.3771],
        [0.2308],
        [0.3942],
        [0.3928],
        [0.2982],
        [0.3375],
        [0.4173],
        [0.35

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
        [0.4020],
        [0.4181],
        [0.3533],
        [0.3962],
        [0.3980],
        [0.1665],
        [0.3818],
        [0.3765],
        [0.2176],
        [0.4062],
        [0.4084],
        [0.2378],
        [0.2177],
        [0.3933],
        [0.2549],
        [0.4166],
        [0.3596],
        [0.3655],
        [0.2541],
        [0.4186],
        [0.4121],
        [0.3800],
        [0.3892],
        [0.3794],
        [0.4326],
        [0.2727],
        [0.3889],
        [0.0926],
        [0.2378],
        [0.4034],
        [0.4083],
        [0.4287],
        [0.3906],
        [0.3918],
        [0.4251],
        [0.3944],
        [0.4130],
        [0.5055],
        [0.3812],
        [0.3839],
        [0.3768],
        [0.3610],
        [0.3423],
        [0.4110],
        [0.3404],
        [0.4017],
        [0.1722],
        [0.4059],
        [0.4211],
        [0.2774],
        [0.3520],
        [0.4344],
        [0.37

...

Streaming output truncated to the last 5000 lines.
        [0.4344],
        [0.4362],
        [0.3086],
        [0.4269],
        [0.4281],
        [0.1044],
        [0.3820],
        [0.3886],
        [0.1462],
        [0.4209],
        [0.3956],
        [0.1949],
        [0.1990],
        [0.3958],
        [0.2192],
        [0.4497],
        [0.3702],
        [0.3533],
        [0.2084],
        [0.4367],
        [0.4449],
        [0.3977],
        [0.4120],
        [0.3902],
        [0.4818],
        [0.2097],
        [0.4135],
        [0.0552],
        [0.1740],
        [0.4185],
        [0.4373],
        [0.4588],
        [0.4140],
        [0.4171],
        [0.4652],
        [0.4242],
        [0.4338],
        [0.5362],
        [0.3563],
        [0.3988],
        [0.3463],
        [0.3698],
        [0.3357],
        [0.4290],
        [0.3320],
        [0.4314],
        [0.1126],
        [0.4159],
        [0.4533],
        [0.2725],
        [0.3566],
        [0.4665],
        [0.39

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
        [0.4411],
        [0.4399],
        [0.3032],
        [0.4328],
        [0.4344],
        [0.0937],
        [0.3803],
        [0.3895],
        [0.1341],
        [0.4212],
        [0.3913],
        [0.1867],
        [0.1935],
        [0.3938],
        [0.2166],
        [0.4566],
        [0.3699],
        [0.3461],
        [0.1982],
        [0.4385],
        [0.4515],
        [0.3993],
        [0.4162],
        [0.3915],
        [0.4935],
        [0.2000],
        [0.4190],
        [0.0491],
        [0.1616],
        [0.4209],
        [0.4432],
        [0.4649],
        [0.4183],
        [0.4215],
        [0.4742],
        [0.4290],
        [0.4344],
        [0.5507],
        [0.3563],
        [0.4008],
        [0.3437],
        [0.3696],
        [0.3315],
        [0.4324],
        [0.3274],
        [0.4380],
        [0.1024],
        [0.4173],
        [0.4591],
        [0.2759],
        [0.3541],
        [0.4757],
        [0.39

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
        [0.3014],
        [0.4350],
        [0.4370],
        [0.0899],
        [0.3796],
        [0.3897],
        [0.1299],
        [0.4208],
        [0.3893],
        [0.1841],
        [0.1914],
        [0.3926],
        [0.2164],
        [0.4593],
        [0.3696],
        [0.3428],
        [0.1944],
        [0.4388],
        [0.4541],
        [0.3997],
        [0.4179],
        [0.3920],
        [0.4981],
        [0.1967],
        [0.4210],
        [0.0469],
        [0.1568],
        [0.4216],
        [0.4455],
        [0.4673],
        [0.4200],
        [0.4230],
        [0.4777],
        [0.4306],
        [0.4342],
        [0.5565],
        [0.3567],
        [0.4015],
        [0.3429],
        [0.3694],
        [0.3297],
        [0.4337],
        [0.3254],
        [0.4407],
        [0.0988],
        [0.4177],
        [0.4611],
        [0.2776],
        [0.3526],
        [0.4796],
        [0.3953],
        [0.3872],
        [0.35

In [20]:
sigmoids[1](input_dim, output_dim)(X_test)

RuntimeError: ignored